In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sep 11 2018
@author: Fernanda Farinelli

Example Response in JSON Format:

|Attribute | Type | Description | Example |
|created_at | String | UTC time when this Tweet was created. | "created_at":"Wed Aug 27 13:08:45 +0000 2008"
|id_str | String | The string representation of the unique identifier for this Tweet. Implementations should use this rather than the large integer in id. | "id_str":"114749583439036416"
|text | String | The actual UTF-8 text of the status update. See twitter-text for details on what characters are currently considered valid. | "text":"Tweet Button, Follow Button, and Web Intents"

"""

'\nCreated on Sep 11 2018\n@author: Fernanda Farinelli\n\nExample Response in JSON Format:\n\n|Attribute | Type | Description | Example |\n|created_at | String | UTC time when this Tweet was created. | "created_at":"Wed Aug 27 13:08:45 +0000 2008"\n|id_str | String | The string representation of the unique identifier for this Tweet. Implementations should use this rather than the large integer in id. | "id_str":"114749583439036416"\n|text | String | The actual UTF-8 text of the status update. See twitter-text for details on what characters are currently considered valid. | "text":"Tweet Button, Follow Button, and Web Intents"\n\n'

In [6]:
"""
truncated	Boolean
Indicates whether the value of the text parameter was truncated, for example, as a result of a retweet exceeding the original Tweet text length limit of 140 characters. Truncated text will end in ellipsis, like this ... Since Twitter now rejects long Tweets vs truncating them, the large majority of Tweets will have this set to false . Note that while native retweets may have their toplevel text property shortened, the original text will be available under the retweeted_status object and the truncated parameter will be set to the value of the original status (in most cases, false ). 
Example: "truncated":true

in_reply_to_status_id_str	String	
Nullable. If the represented Tweet is a reply, this field will contain the string representation of the original Tweet’s ID. 
Example: "in_reply_to_status_id_str":"114749583439036416"

n_reply_to_user_id_str	String	
Nullable. If the represented Tweet is a reply, this field will contain the string representation of the original Tweet’s author ID. This will not necessarily always be the user directly mentioned in the Tweet. Example:

"in_reply_to_user_id_str":"819797"
in_reply_to_screen_name	String	
Nullable. If the represented Tweet is a reply, this field will contain the screen name of the original Tweet’s author. Example:

"in_reply_to_screen_name":"twitterapi"

"""

'\ntruncated\tBoolean\nIndicates whether the value of the text parameter was truncated, for example, as a result of a retweet exceeding the original Tweet text length limit of 140 characters. Truncated text will end in ellipsis, like this ... Since Twitter now rejects long Tweets vs truncating them, the large majority of Tweets will have this set to false . Note that while native retweets may have their toplevel text property shortened, the original text will be available under the retweeted_status object and the truncated parameter will be set to the value of the original status (in most cases, false ). \nExample: "truncated":true\n\nin_reply_to_status_id_str\tString\t\nNullable. If the represented Tweet is a reply, this field will contain the string representation of the original Tweet’s ID. \nExample: "in_reply_to_status_id_str":"114749583439036416"\n\nn_reply_to_user_id_str\tString\t\nNullable. If the represented Tweet is a reply, this field will contain the string representation o

In [3]:
# Constants mongoDB
server = 'localhost'
port = 27017
dbname = 'opladyn'
collectionname = 'caseMarielle'

# Constants twitter
consumer_key = 'rWxMBAKB6IgXNVrsXkisVebMU'
consumer_secret = 'xRFc47WcbcgoZYb26OHfMXakda7UFIQMvZw2WftbOsAXpBRKMS'
access_token_key = '1028301224519589889-hrC4g78bb0PJ9RiEbusi657it517wf'
access_token_secret = 'hhwamERYlZUJuLQwWnJtvWvqpB5hFCyNBOCiDVDrT5xIL'

#filename = "C://Users//unesp//ProjetoOpLaDyn//inputs//twitter_tokens_pt.txt"
fileInput = "C://Users//unesp//ProjetoOpLaDyn//inputs//twitter_tokens_Marielle_pt.txt"
folderOutput = 'C:\\Users\\unesp\\ProjetoOpLaDyn\\outputs\\'
filename = "tweetsKeywordsCaseMarielle_pt_"

In [4]:
# Import modules
#import tweepy #https://github.com/tweepy/tweepy
import csv
import json
#import time
#import unicodedata

import tweepy
from tweepy import OAuthHandler
#import sys
import pymongo
from pymongo import MongoClient


In [16]:
# Helper functions

# Authenticate
def auth(consumer_key, consumer_secret, access_token_key, access_token_secret):
	# Authenticate
	auth = OAuthHandler(consumer_key, consumer_secret)
	auth.set_access_token(access_token_key, access_token_secret)
 
	# Connect the api
	api = tweepy.API(auth)
	# Return
	return(api)

# Download tweets with hashtag or keyword
def get_tweets(collection,api,tokens):
    global folderOutput
    number = 1000
    # The search term you want to find (hashtag)
    alltweets = []
    i=0
    stored = 0
    while i < len(tokens):

        hashtag = tokens[i].strip()
        cursor = tweepy.Cursor(api.search, q=hashtag,rpp=100,include_entities=True).items(number)        
        # For each element, send to mongo
        for c in cursor:
            cJson = c._json
            # Check if in Mongo
            #res = checkMongo(collection, hashtag, cJson["id"])
            print('id twitter:',cJson["id"],"-",cJson["id_str"])
            res = checkMongo(collection, cJson["id"])
            # If false . . . result not stored in database
            if res == False:
                #saveToMongo(collection, hashtag, cJson)
                saveToMongo(collection,cJson)
                alltweets.extend(cJson)
                stored = stored + 1
                
    #preparing file to write
    timestr = time.strftime("%Y%m%d-%H%M%S")
    #print(timestr)
    writeJSON(alltweets,timestr)
    
    return("Found a total of "+stored+" tweets")
'''    
# Download tweets with hashtag
def dTweets(collection, api, hashtag, number):
	# Get cursor
	curs = tweepy.Cursor(api.search, q=hashtag).items(number)
	# For each element, send to mongo
	for c in curs:
		cJson = c._json
    	# Check if in Mongo
		res = checkMongo(collection, hashtag, cJson["id"])
		# If true . . .
		if res == True:
			return("Found duplicate")
		else:
			saveToMongo(collection, hashtag, cJson)
	# Return
	return("All done")
'''
#Creating a connection to database server
def getServerConnection():
    global server, port
    return pymongo.MongoClient(server,port)

#Defining a database in such server
def getDBConnection(con):
    global dbname
    return con[dbname]

#Connecting to a database collection
def getCollection(dbs):
    global collectionname
    return dbs[collectionname]

def checkMongo(collection, id):
    # Check
    #print("checking...")
    if collection.find_one({"id": id}):
        found = True
        print("checked: True")
    else:
        found = False
        #print("checked: False")

    return found
'''
def checkMongo(collection, hashtag, id):
	# Connect
	client = MongoClient()
	db = client[collection]
	# Check
	if db[hashtag].find_one({"id": id}):
		found = True
	else:
		found = False
    # Disconnect
	client.close()
    # Return
	return found
'''
# Save to mongo
def saveToMongo(collection, result):
    # Insert
    #print("inserting...")
    collection.insert_many(result)
    
'''
def saveToMongo(collection, hashtag, result):
	client = MongoClient()
	db = client[collection]
    # Insert
	db[hashtag].insert(result)
    # Close
	client.close()
'''
# get tokens
def get_tokens():
    global fileInput

    with open(fileInput, 'r') as twitter_tokens:
        tokens = twitter_tokens.read().split(',')
        print('Keywords list:',tokens)
    return tokens
    
def writeJSON(alltweets,timestr):
    global filename, folderOutput
    file = folderOutput + filename + timestr
    #write tweet objects to JSON
    file = open(file + '.json','w')
    print ("Writing tweet objects to JSON please wait...")
    for status in alltweets:
        json.dump(status._json,file,sort_keys = True,indent = 4)
    #close the file
    print ("Done with JSON")
    file.close()
    
# Main
def main():
    # global arguments
    global consumer_key, consumer_secret, access_token_key, access_token_secret
    
    # Authenticate twitter
    api = auth(consumer_key, consumer_secret, access_token_key, access_token_secret)

    #Connect MongoDB database/collection
    connection = getServerConnection()
    database = getDBConnection(connection)
    collection = getCollection(database)

    # Getting hashtag or keywords from input file
    tokens = get_tokens()

    # Get tweets and save in a mongoDB
    print (get_tweets(collection,api,tokens))
    

In [18]:
# Run

if __name__ == "__main__":
    main()

Keywords list: ['"Marielle e Anderson"', '"Marielle Franco"', 'marielle', '"quem matou marielle"', '#Marielle', '#MarielleFranco', '#MariellePresente', '#JustiçaParaMarielleEAnderson', '"Marielle Presente"', '#QuemMatouMarielle\n']
id twitter: 1040337617026510848 - 1040337617026510848
checking...
checked: False
inserting...
id twitter: 1040337333726400513 - 1040337333726400513
checking...
checked: False
inserting...
id twitter: 1040336903990636550 - 1040336903990636550
checking...
checked: False
inserting...
id twitter: 1040336849330470912 - 1040336849330470912
checking...
checked: False
inserting...
id twitter: 1040336447662960641 - 1040336447662960641
checking...
checked: False
inserting...
id twitter: 1040335646412427265 - 1040335646412427265
checking...
checked: False
inserting...
id twitter: 1040335072036102145 - 1040335072036102145
checking...
checked: False
inserting...
id twitter: 1040334916930678795 - 1040334916930678795
checking...
checked: False
inserting...
id twitter: 1040

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:109: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


id twitter: 1040331812122316802 - 1040331812122316802
checking...
checked: False
inserting...
id twitter: 1040331633503752192 - 1040331633503752192
checking...
checked: False
inserting...
id twitter: 1040331218770911232 - 1040331218770911232
checking...
checked: False
inserting...
id twitter: 1040331089930272770 - 1040331089930272770
checking...
checked: False
inserting...
id twitter: 1040330753756864512 - 1040330753756864512
checking...
checked: False
inserting...
id twitter: 1040330230815227905 - 1040330230815227905
checking...
checked: False
inserting...
id twitter: 1040330109914423296 - 1040330109914423296
checking...
checked: False
inserting...
id twitter: 1040329984806666245 - 1040329984806666245
checking...
checked: False
inserting...
id twitter: 1040329629708500993 - 1040329629708500993
checking...
checked: False
inserting...
id twitter: 1040329539782692864 - 1040329539782692864
checking...
checked: False
inserting...
id twitter: 1040328338433294336 - 1040328338433294336
checki

id twitter: 1040308196731248645 - 1040308196731248645
checking...
checked: True
id twitter: 1040307682165628931 - 1040307682165628931
checking...
checked: True
id twitter: 1040307244141817861 - 1040307244141817861
checking...
checked: True
id twitter: 1040307137812094977 - 1040307137812094977
checking...
checked: True
id twitter: 1040307016974131202 - 1040307016974131202
checking...
checked: True
id twitter: 1040306728838070272 - 1040306728838070272
checking...
checked: True
id twitter: 1040306630313889793 - 1040306630313889793
checking...
checked: True
id twitter: 1040306566149414914 - 1040306566149414914
checking...
checked: True
id twitter: 1040306207460864001 - 1040306207460864001
checking...
checked: True
id twitter: 1040305507737776128 - 1040305507737776128
checking...
checked: True
id twitter: 1040305406378237952 - 1040305406378237952
checking...
checked: True
id twitter: 1040305292024655873 - 1040305292024655873
checking...
checked: True
id twitter: 1040305104002445315 - 104030

id twitter: 1040290444251414530 - 1040290444251414530
checking...
checked: True
id twitter: 1040290220569182210 - 1040290220569182210
checking...
checked: True
id twitter: 1040290155922444288 - 1040290155922444288
checking...
checked: True
id twitter: 1040290130974703616 - 1040290130974703616
checking...
checked: True
id twitter: 1040290079447687169 - 1040290079447687169
checking...
checked: True
id twitter: 1040289918029848576 - 1040289918029848576
checking...
checked: True
id twitter: 1040289856465920001 - 1040289856465920001
checking...
checked: True
id twitter: 1040289721795141633 - 1040289721795141633
checking...
checked: True
id twitter: 1040289430014251008 - 1040289430014251008
checking...
checked: True
id twitter: 1040289251559124992 - 1040289251559124992
checking...
checked: True
id twitter: 1040289091861004290 - 1040289091861004290
checking...
checked: True
id twitter: 1040288946310311936 - 1040288946310311936
checking...
checked: True
id twitter: 1040288886440771584 - 104028

...

id twitter: 1040272433213853696 - 1040272433213853696
checking...
checked: True
id twitter: 1040272116535451648 - 1040272116535451648
checking...
checked: True
id twitter: 1040272082318307331 - 1040272082318307331
checking...
checked: True
id twitter: 1040272063691411457 - 1040272063691411457
checking...
checked: True
id twitter: 1040271984519786496 - 1040271984519786496
checking...
checked: True
id twitter: 1040271920237817857 - 1040271920237817857
checking...
checked: True
id twitter: 1040271805053906944 - 1040271805053906944
checking...
checked: True
id twitter: 1040271796837212164 - 1040271796837212164
checking...
checked: True
id twitter: 1040271698644361216 - 1040271698644361216
checking...
checked: True
id twitter: 1040271522311692288 - 1040271522311692288
checking...
checked: True
id twitter: 1040271470725931008 - 1040271470725931008
checking...
checked: True
id twitter: 1040271119461302277 - 1040271119461302277
checking...
checked: True
id twitter: 1040271046191067137 - 104027

id twitter: 1040263709388668928 - 1040263709388668928
checking...
checked: True
id twitter: 1040263691038543872 - 1040263691038543872
checking...
checked: True
id twitter: 1040263656628543488 - 1040263656628543488
checking...
checked: True
id twitter: 1040263634440671234 - 1040263634440671234
checking...
checked: True
id twitter: 1040263576395702274 - 1040263576395702274
checking...
checked: True
id twitter: 1040263529524281349 - 1040263529524281349
checking...
checked: True
id twitter: 1040263525782958086 - 1040263525782958086
checking...
checked: True
id twitter: 1040263433579626496 - 1040263433579626496
checking...
checked: True
id twitter: 1040263428009537538 - 1040263428009537538
checking...
checked: True
id twitter: 1040263414004768769 - 1040263414004768769
checking...
checked: True
id twitter: 1040263377656995843 - 1040263377656995843
checking...
checked: True
id twitter: 1040263259931242496 - 1040263259931242496
checking...
checked: True
id twitter: 1040263206160224256 - 104026

id twitter: 1040217468164296707 - 1040217468164296707
checking...
checked: True
id twitter: 1040217326883418117 - 1040217326883418117
checking...
checked: True
id twitter: 1040217169584373760 - 1040217169584373760
checking...
checked: True
id twitter: 1040216742465888256 - 1040216742465888256
checking...
checked: True
id twitter: 1040216165027651585 - 1040216165027651585
checking...
checked: True
id twitter: 1040215500515622912 - 1040215500515622912
checking...
checked: True
id twitter: 1040214918396633088 - 1040214918396633088
checking...
checked: True
id twitter: 1040214822544130049 - 1040214822544130049
checking...
checked: True
id twitter: 1040214755821215744 - 1040214755821215744
checking...
checked: True
id twitter: 1040214659389956097 - 1040214659389956097
checking...
checked: True
id twitter: 1040213818532671488 - 1040213818532671488
checking...
checked: True
id twitter: 1040212914509103105 - 1040212914509103105
checking...
checked: True
id twitter: 1040212812323274753 - 104021

id twitter: 1040186011123372032 - 1040186011123372032
checking...
checked: True
id twitter: 1040185983222915072 - 1040185983222915072
checking...
checked: True
id twitter: 1040185495605706752 - 1040185495605706752
checking...
checked: True
id twitter: 1040185412839460864 - 1040185412839460864
checking...
checked: True
id twitter: 1040185385349992449 - 1040185385349992449
checking...
checked: True
id twitter: 1040185358716215296 - 1040185358716215296
checking...
checked: True
id twitter: 1040184974509592576 - 1040184974509592576
checking...
checked: True
id twitter: 1040184969958752256 - 1040184969958752256
checking...
checked: True
id twitter: 1040184762252632064 - 1040184762252632064
checking...
checked: True
id twitter: 1040184715196751872 - 1040184715196751872
checking...
checked: True
id twitter: 1040184607717699584 - 1040184607717699584
checking...
checked: True
id twitter: 1040184241236140033 - 1040184241236140033
checking...
checked: True
id twitter: 1040184176727732224 - 104018

TweepError: Twitter error response: status code = 429